In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd

## Data Preprocessing

In [3]:
data_path = 'batman.txt'

In [4]:
input_texts = []
target_texts = []

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

    for line in lines:
        if line.strip():  # Skip empty lines
            parts = line.split('Q:')
            if len(parts) >= 2:
                input_text = parts[1].split('A:')[0].strip()
                target_text_parts = parts[1].split('A:')
                target_text = 'A:'.join(target_text_parts[1:]).strip() if len(target_text_parts) > 1 else ''
                input_texts.append(input_text)
                target_texts.append(target_text)

In [5]:
len(input_texts)

500

In [6]:
zippedList =  list(zip(input_texts, target_texts))
lines = pd.DataFrame(zippedList, columns = ['input' , 'output']) 

In [7]:
lines.head()

,input,output
0,Who created Batman?,Batman was created by Bob Kane and Bill Finger.
1,In which year did Batman first appear in comics?,Batman first appeared in Detective Comics #27 ...
2,What is Batman's secret identity?,"Batman's secret identity is Bruce Wayne, a wea..."
3,"Where is Gotham City, the home of Batman, loca...",Gotham City is a fictional city in the DC Comi...
4,Who is Batman's most famous sidekick?,"Batman's most famous sidekick is Robin, who ha..."


## Preparing input data for the Encoder

In [8]:
input_lines = list()
for line in lines.input:
    input_lines.append( line ) 

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( input_lines ) 
tokenized_input_lines = tokenizer.texts_to_sequences( input_lines ) 

length_list = list()
for token_seq in tokenized_input_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
#print( 'Input max length is {}'.format( max_input_length ))

padded_input_lines = preprocessing.sequence.pad_sequences( tokenized_input_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_input_lines )
#print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

input_word_dict = tokenizer.word_index
num_input_tokens = len( input_word_dict )+1
#print( 'Number of Input tokens = {}'.format( num_input_tokens))


## Preparing input data for the Decoder

In [9]:
output_lines = list()
for line in lines.output:
    output_lines.append( '<START> ' + line + ' <END>' )  

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( output_lines ) 
tokenized_output_lines = tokenizer.texts_to_sequences( output_lines ) 

length_list = list()
for token_seq in tokenized_output_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
#print( 'Output max length is {}'.format( max_output_length ))

padded_output_lines = preprocessing.sequence.pad_sequences( tokenized_output_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_output_lines )
#print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

output_word_dict = tokenizer.word_index
num_output_tokens = len( output_word_dict )+1
#print( 'Number of Output tokens = {}'.format( num_output_tokens))


## Preparing target data for the Decoder 

In [10]:
decoder_target_data = list()
for token_seq in tokenized_output_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
padded_output_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_output_lines = utils.to_categorical( padded_output_lines , num_output_tokens )
decoder_target_data = np.array( onehot_output_lines )
#print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

## Defining the Model

In [11]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_input_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_output_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_output_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, None, 256)         │         222,720 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ not_equal (NotEqual)          │ (None, None)              │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, None, 256)         │         393,728 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 256), (None,      │         525,312 │ embedding[0][0],           │
│                               │ 256), (None, 256)]        │                 │ not_equal[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, None, 256),       │         525,312 │ embedding_1[0][0],         │
│                               │ (None, 256), (None, 256)] │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, None, 1538)        │         395,266 │ lstm_1[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,062,338 (7.87 MB)

 Trainable params: 2,062,338 (7.87 MB)

 Non-trainable params: 0 (0.00 B)

## Training

In [16]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=256, epochs=250) 
model.save( 'model.h5' ) 

Epoch 1/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 497ms/step - accuracy: 0.0349 - loss: 7.3356
Epoch 2/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 492ms/step - accuracy: 0.1938 - loss: 7.3120
Epoch 3/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 507ms/step - accuracy: 0.1934 - loss: 7.2528
Epoch 4/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 493ms/step - accuracy: 0.0982 - loss: 7.0228
Epoch 5/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 490ms/step - accuracy: 0.0753 - loss: 6.4241
Epoch 6/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 509ms/step - accuracy: 0.0624 - loss: 5.8379
Epoch 7/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 494ms/step - accuracy: 0.0614 - loss: 5.3862
Epoch 8/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 482ms/step - accuracy: 0.0683 - loss: 5.0825
Epoch 9/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 495ms/step - accuracy: 0.0784 - loss: 4.9552
Epoch 10/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 489ms/step - accuracy: 0.0826 - loss: 4.8925
Epoch 11/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 495ms/step - accuracy: 0.1379 - loss: 4.8841
Epoch 12/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 494ms/step - accuracy:

## Inference models

In [17]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(256,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


In [18]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( input_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')


In [19]:
enc_model, dec_model = make_inference_models()

enc_model.save('enc_model.h5')
dec_model.save('dec_model.h5')
model.save('model.h5')

## NLP Techniques
- They have been reused from Project 1.

## Chatbot

In [21]:
import json
import os
import re
import sys
import nltk
import numpy as np

from DocumentRetrieval import DocumentRetrieval as DRM
from Question import Question as PQ

nltk.download('maxent_ne_chunker')
nltk.download('words')

username_file = "username_history.json"
username_history = {}

if os.path.exists(username_file) and os.stat(username_file).st_size > 0:
    with open(username_file, "r") as f:
        try:
            username_history = json.load(f)
        except json.JSONDecodeError:
            username_history = {}
else:
    username_history = {}

username = input("Bot: Please enter your username: ")
if username in username_history:
    print(f"Bot: Welcome back, {username}!")
else:
    likes = input("Bot: Which is your favourite Batman movie?\n")
    dislikes = input("Bot: What aspects of Batman's character or stories do you dislike?\n")
    username_history[username] = {"likes": likes, "dislikes": dislikes}
    with open(username_file, "w") as f:
        json.dump(username_history, f)
    print(f"Bot: Welcome, {username}!")

if len(sys.argv) == 1:
    print("Bot: I need some reference to answer your question")
    print("Bot: Please! Rerun me")
    print("Bot: Thanks! Bye")
    exit()

with open('output.txt', 'w', encoding="utf-8") as f:
    for text in target_texts:
        f.write(text + '\n\n')
datasetName = 'output.txt'
# Loading Dataset
datasetFile = open(datasetName,"r", encoding="utf-8")

paragraphs = []
for para in datasetFile.readlines():
    if(len(para.strip()) > 0):
        paragraphs.append(para.strip())

# Processing Paragraphs
drm = DRM(paragraphs, True, True)

print(f"Bot: Hi {username}! Ask me questions regarding Batman")
print("Bot: Enter 'bye' to exit")

# Greet Pattern
greetPattern = re.compile("^\ *((hi+)|((good\ )?morning|evening|afternoon)|(he((llo)|y+)))\ *$",re.IGNORECASE)

isActive = True

isActive = True
while isActive:
    userQuery = input("User: ")
    if not len(userQuery) > 0:
        print("Bot: You need to ask something")
    elif userQuery.strip().lower() == "bye":
        response = "Bye Bye!"
        isActive = False
    else:
        try:
            states_values = enc_model.predict(str_to_tokens(userQuery))
            empty_target_seq = np.zeros((1, 1))
            empty_target_seq[0, 0] = output_word_dict['start']
            stop_condition = False
            decoded_translation = ''
            while not stop_condition:
                dec_outputs, h, c = dec_model.predict([empty_target_seq] + states_values)
                sampled_word_index = np.argmax(dec_outputs[0, -1, :])
                sampled_word = None
                for word, index in output_word_dict.items():
                    if sampled_word_index == index:
                        decoded_translation += ' {}'.format(word)
                        sampled_word = word
                if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
                    stop_condition = True
                empty_target_seq = np.zeros((1, 1))
                empty_target_seq[0, 0] = sampled_word_index
                states_values = [h, c]
            response = decoded_translation.replace(' end', '')
        except Exception as e:
            # Process Question using other conditions
            pq = PQ(userQuery, True, False, True)
            # Get Response From Bot
            response = drm.query(pq)

    print("Bot:", response)
    print()


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Aaryan\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Aaryan\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Bot: Please enter your username:  Ankit


Bot: Welcome back, Ankit!
Bot: Hi Ankit! Ask me questions regarding Batman
Bot: Enter 'bye' to exit


User:  who created batman


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Bot:  batman was created by bob kane and bill finger



...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Bot:  the batman villain who can control plants is pamela isley also known as poison ivy



User:  What is the name of the serial killer obsessed with the number 13


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Bot:  the serial killer obsessed with the number 13 is known as the calendar man



User:  bye


Bot: Bye Bye!

